In [ ]:
# For ! installing stuff

! jt -t oceans16

In [1]:
# PART 2 : Converting to a Gym Enviroment

from sre_parse import State
from vizdoom import *
from gym import Env # Import envoriment base class from OpenAI Gym
from gym.spaces import Discrete, Box # Import gym spaces
import cv2 # Import opencv to greyscale stuff
import numpy as np
from matplotlib import pyplot as plt



# Imports for part 3
import torch # pyTorch
import os # for file navigation
from stable_baselines3.common.callbacks import BaseCallback # import callback class from stablebaselines 3
from stable_baselines3.common import env_checker # For checking if enviroment is in correct format
import stable_baselines3

# Imports for Part 4
from stable_baselines3 import PPO # Import PPO for training


C:\Users\dell\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [7]:
# Create VizDOOM OpenAI Gym Enviroment
class VizDoomGym(Env):

    def __init__(self, render = False, config='ViZDoom/scenarios/deadly_corridor_s1.cfg'):
        super().__init__() # Inherit from "Env" class ???
        # TODO: Learn more about OpenAI Gym

        # Set up game
        self.game = vizdoom.DoomGame() # TODO: IDK if "vizdoom." does anything
        self.game.load_config(config)

       # Render frame logic
       # Rendering takes a lot of computing and we don't always want to see shit cause we don't care
        if(render == False):
            self.game.set_window_visible(False) # Don't pop up that window - we don't want to see it
        else:
            self.game.set_window_visible(True) # Show us the window

        # Start game after we know whether to render or not
        self.game.init()

        # Set up action space and observation space
        # TODO: I don't really get what these are
        self.observation_space = Box(low = 0, high = 255, shape = (100, 160, 1), dtype=np.uint8)
        self.action_space = Discrete(7) # 7 actions we can take now
        
        # Game variables are HEALTH, DAMAGE_TAKEN, HITCOUNT, SELECTED_WEAPON_AMMO
        self.damage_taken = 0
        self.hitcount = 0 
        # Hitcount can be dodgy - it works with ALL monsters hit not even by our agent
        # Damagecount can be better sometimes
        # Look up ViZDoom game variables
        self.ammo = 52
 

    def step(self, action): # How we take a step in the enviroment

        # Specify action and take step
        actions = np.identity(7) # 7 actions as in config file, represented as identity matrices
        movement_reward = self.game.make_action(actions[action], 4) 
        # Make the action and get the reward, 4 = frameskip parameter
        # TODO: I don't really get movement reward
        
        reward = 0
        # Get other stuff we need to return
        if (self.game.get_state()):
            state = self.game.get_state().screen_buffer # The next frame of the game
            state = self.greyscale(state) # Does the grayscaling and resizing of the image, implemented in greyscale() method
            
            # REWARD SHAPING
            
            game_variables = self.game.get_state().game_variables
            health, damage_taken, hitcount, ammo = game_variables
            
            # We want to calculate changes - DELTAS
            damage_taken_delta = self.damage_taken - damage_taken
            self.damage_taken = damage_taken
            hitcount_delta = hitcount - self.hitcount # Remember that a hit kills
            self.hitcount = hitcount
            ammo_delta = ammo - self.ammo # You don't wannt to miss shots
            self.ammo = ammo
            
            # The tutorial man says these weights work well - just taking his word for it
            reward = movement_reward + damage_taken_delta*10 + hitcount_delta*200 + ammo_delta*5
            
            # END OF REWARD SHAPING
            
            info = ammo 
        else: # This logic in case we are finished and there is no next frame - would throw an error otherwise
            # Just returns zeroes for shit
            state = np.zeros(self.observation_space.shape)
            info = 0

        info = {"info":info}

        done = self.game.is_episode_finished() # Whether or not the thing is finished

        return state, reward, done, info

    def render(): # Predifined in Vizdoom but needed to be openAI superclass or smth
        pass

    def reset(self): # What happens when we start a new game
        self.game.new_episode() # Make a new game
        state = self.game.get_state().screen_buffer # Next frame

        return self.greyscale(state) # Return next frame, greyscaled

    def greyscale(self, observation): # Greyscale and resize the game frame, get rid of the bottom bit too
        # Applied in step() and reset()
        # Gets rid of color channel i.e. the 3
        # TODO: Maybe figure out how this works
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY) # Making shit gray - idk how this works

        # Compresses frame down ???
        resize = cv2.resize(gray, (160, 100), interpolation = cv2.INTER_CUBIC) # Reiszes image and scales it down - so we have more pixels to process
        state = np.reshape(resize, (100, 160, 1)) # 

        return state

    def close(self): # Close down the game so it's not floating
        self.game.close()

# Enviroment is now set up

In [3]:
# TUTORIAL 3

# Setup callback

# Standard training and logging callback
# Used for saving the model in case shit goes wrong
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose = 1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok = True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True


# Directories for saving and logging shit
CHECKPOINT_DIR = './train/train_corridor' # Checkpoint directory for saving trained reinforcement learning models
LOG_DIR = './logs/log_corridor' # 

# Create instance of train and logging callback

callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)
# check_freq = 10000 means that after every 10000 steps of training our model we're going to save a version of those pyTorch weights for our reinforcement learning agent (can be re-loaded)


In [14]:
# Tutorial 4 - Train the RL Model usign curriculum learning


env = VizDoomGym(render = False, config='ViZDoom/scenarios/deadly_corridor_s1.cfg')

model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.00001, n_steps=8192, clip_range=0.10, gamma=0.95, gae_lambda=0.9)
# Look at like 2:55 for epxlination of these hyperparameter values
# n_steps = how many time frames I'm going to be passing in total as part of one traiing run
# larger = more information we're going to be passing per epich
# Should be a multiple (fuction) of 64 - 2:58


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [15]:
# Enviroment checker

env_checker.check_env(env) # Runs with no errors if env is all good

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
# Runs the PPO training model

model.learn(total_timesteps=100_000, callback=callback )

# Get tensorboard graphs with (in ppo training folder) "tensorboard --logrid=."
# He explains tensorboard graphs at 1:42

# Agent is now trained

In [ ]:
# Tutorial 5 - Testing the Agent
import time
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
model.load('./train/train_defend/best_model_70000') # Reload model from disk
##PPO.load() works too

env = VizDoomGym(render = True)
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes = 10)
# "_" is used when something you don't care about is returned
# pass in model, enviroment and amount of episodes we want to evaluate the policy for

print(mean_reward)

In [ ]:
# This is a lot like what we did in the very beginning i.e. the initial tutorial

for episode in range(5): # 5 games
    obs = env.reset() # reset enviroment, store game frame in "observations" variable
    done = False
    total_reward= 0
    while not done:
        action, _ = model.predict(obs) # model predicts what ation to take
        obs, reward, done, info = env.step(action) # actually take the predicted action
        time.sleep(0.25)
        total_reward += reward
    print("Total reward for episode {} is {}".format(episode, total_reward))
    time.sleep(2)
    
# 1:50

In [ ]:
# Tutorial 7 - Deadly Corridor

"""
Stuff you can do to train agents to play harder games, 2 techniques usually work pretty well
    > Reward shaping - helps ease the sparse reward problem
        - So like if you're doing a maze and the only reward is finishing the maze, the agent
          gets fuck all info from that
        - Reward shaping gives incrimental feedback and incrimental rewards for completing 
          sub-goals i.e. getting closer to the end of the maze rather than just completing it
    > Curriculum learning - train the agent in easier enviroment and progressively increase 
      the complexity and/or difficulty so the agent learns to play in a real-world scenario
          - For Doom, we can do this by changing the difficulty setting int the config file

"""

# Look at "CLIENT CONVERSATION 6" for an explination of this

# First we do reward shaping, adding new parameters

In [ ]:
# Train our Model using Curriculum Learning 

In [13]:
print(game.get_state())

None


In [ ]:
# TODO: LOOK INTO OPTUNA